In [5]:
%load_ext autoreload
%autoreload 2

In [107]:
import torch
import random
import numpy as np
from HP import HawkesProcess

In [108]:
#K = 3
N = 20
D = 4
C = 10
#T = 10
mu = torch.rand(C)
A = torch.rand(C, C, D)
basis_fs = [lambda x: torch.exp(- x**2 / k ) for k in range(D)]
s = []
Ts = []

for n in range(N):
    Tn = 3
    Mn = random.randint(10, 20)
    Ts.append(Tn)
    ts = torch.FloatTensor([random.random() * Tn for _ in range(Mn)]).sort()[0]
    cs = torch.FloatTensor([random.randint(0, C - 1) for _ in range(Mn)])
    s.append(torch.stack([ts, cs], -1))

In [109]:
mu

tensor([0.3294, 0.3215, 0.8736, 0.6446, 0.2780, 0.0246, 0.5077, 0.5639, 0.6300,
        0.9202])

In [110]:
hp = HawkesProcess(mu, A, basis_fs, Ts, s)

In [111]:
lambdat = hp.intensity(ts=torch.FloatTensor(np.linspace(0, 3, 15)))

In [112]:
lambdat.shape

torch.Size([20, 15, 10])

In [113]:
lambdat.min()

tensor(0.0246)

In [114]:
lambdas = [hp.intensity(sn[:, 0], cs=sn[:, 1]) for sn in hp.s]

In [125]:
lambdas[0][:, 0]

tensor([0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215,
        0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215, 0.3215,
        0.3215, 0.3215])

In [116]:
len(lambdas), lambdas[0].shape

(20, torch.Size([20, 18]))

In [117]:
ll = hp.log_likelihood()

In [118]:
ll.shape

torch.Size([20])

In [119]:
ll

tensor([-690.5427, -691.2208, -348.4326, -523.4294, -478.1431, -680.6556,
        -505.7257, -721.5681, -448.1017, -479.4576, -359.3188, -483.9387,
        -639.1586, -666.1451, -773.5902, -398.5319, -603.0493, -416.3192,
        -477.3661, -631.4401])

In [120]:
s = []
Ts = 3

for n in range(N):
    Mn = random.randint(10, 20)
    ts = torch.FloatTensor([random.random() * Ts for _ in range(Mn)]).sort()[0]
    cs = torch.FloatTensor([random.randint(0, C - 1) for _ in range(Mn)])
    s.append(torch.stack([ts, cs], -1))

In [121]:
hp = HawkesProcess(mu, A, basis_fs, Ts, s)

In [122]:
ll = hp.log_likelihood()

In [123]:
ll

tensor([-573.3574, -572.8993, -402.5913, -539.5865, -550.7906, -702.5536,
        -642.9664, -450.0692, -525.2098, -635.9698, -392.0526, -404.1013,
        -673.3016, -616.3384, -408.4383, -364.3509, -736.5417, -388.2557,
        -416.1881, -623.4576])